In [1]:
import csv
import datetime
from sklearn import preprocessing
import numpy as np
from scipy import sparse
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import linear_model
from sklearn import naive_bayes
import xgboost as xgb
import time

In [2]:
def load(file):
    #open file
    inF = open(file,'rb')
    rdr = csv.reader(inF,delimiter=',')

    #read column names
    cols = rdr.next()
    print cols

    #extract data
    dat = []
    for l in rdr:
        dat.append(l)
    return dat

dat = load('data/train.csv')
test = load('data/test.csv')

['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']
['Id', 'Store', 'DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']


In [3]:
#extract target var
Y = np.array([float(i[3]) for i in dat])

#drop sales,customers variables as not in test set
dat = [dat[i][:3] + dat[i][5:] for i in range(len(dat))]

#drop id from test set
test = [test[i][1:] for i in range(len(test))]

print dat[0]
print test[0]

['1', '5', '2015-07-31', '1', '1', '0', '1']
['1', '4', '2015-09-17', '1', '1', '0', '0']


In [4]:
#convert dates to dt
for i in range(len(dat)):
    d = datetime.datetime.strptime(dat[i][2],'%Y-%m-%d')
    dat[i] = dat[i][:2] + dat[i][3:] + [str(d.year),str(d.month),str(d.day)]

for i in range(len(test)):
    d = datetime.datetime.strptime(test[i][2],'%Y-%m-%d')
    test[i] = test[i][:2] + test[i][3:] + [str(d.year),str(d.month),str(d.day)]

In [5]:
print dat[0]
print test[0]

['1', '5', '1', '1', '0', '1', '2015', '7', '31']
['1', '4', '1', '1', '0', '0', '2015', '9', '17']


In [6]:
#encode string to int
le1,le2,le3,le4,le5,le6,le7,le8,le9 = preprocessing.LabelEncoder(),preprocessing.LabelEncoder(),preprocessing.LabelEncoder(),preprocessing.LabelEncoder(),preprocessing.LabelEncoder(),preprocessing.LabelEncoder(),preprocessing.LabelEncoder(),preprocessing.LabelEncoder(),preprocessing.LabelEncoder()
le1.fit([i[0] for i in dat])
le2.fit([i[1] for i in dat])
le3.fit([i[2] for i in dat])
le4.fit([i[3] for i in dat])
le5.fit([i[4] for i in dat])
le6.fit([i[5] for i in dat])
le7.fit([i[6] for i in dat])
le8.fit([i[7] for i in dat])
le9.fit([i[8] for i in dat])

print 'stores ',le1.classes_
print 'dow ',le2.classes_
print 'Open ',le3.classes_
print 'Promo ',le4.classes_
print 'StateHoliday ',le5.classes_
print 'SchoolHoliday ',le6.classes_

#transform train data
X_cat1 = le1.transform([i[0] for i in dat])
X_cat2 = le2.transform([i[1] for i in dat])
X_cat3 = le3.transform([i[2] for i in dat])
X_cat4 = le4.transform([i[3] for i in dat])
X_cat5 = le5.transform([i[4] for i in dat])
X_cat6 = le6.transform([i[5] for i in dat])
X_cat7 = le7.transform([i[6] for i in dat])
X_cat8 = le8.transform([i[7] for i in dat])
X_cat9 = le9.transform([i[8] for i in dat])
X_cat = [[X_cat1[i],X_cat2[i],X_cat3[i],X_cat4[i],X_cat5[i],X_cat6[i],X_cat7[i],X_cat8[i],X_cat9[i]] for i in range(len(X_cat1))]

#fix up test data i[2] contains the 'open' variable which in the test set has some unknown values
#here we assume if open is unknown then the shop is open
for i in test:
    if i[2]=='':
        i[2]='1'

#transform test data
test_cat1 = le1.transform([i[0] for i in test])
test_cat2 = le2.transform([i[1] for i in test])
test_cat3 = le3.transform([i[2] for i in test])
test_cat4 = le4.transform([i[3] for i in test])
test_cat5 = le5.transform([i[4] for i in test])
test_cat6 = le6.transform([i[5] for i in test])
test_cat7 = le7.transform([i[6] for i in test])
test_cat8 = le8.transform([i[7] for i in test])
test_cat9 = le9.transform([i[8] for i in test])
test_cat = [[test_cat1[i],test_cat2[i],test_cat3[i],test_cat4[i],test_cat5[i],test_cat6[i],test_cat7[i],test_cat8[i],test_cat9[i]] for i in range(len(test_cat1))]


stores  ['1' '10' '100' ..., '997' '998' '999']
dow  ['1' '2' '3' '4' '5' '6' '7']
Open  ['0' '1']
Promo  ['0' '1']
StateHoliday  ['0' 'a' 'b' 'c']
SchoolHoliday  ['0' '1']


In [7]:
X_cat[0]

[0, 4, 1, 1, 0, 1, 2, 9, 24]

In [8]:
#create dummy vars
enc = preprocessing.OneHotEncoder(sparse=True)
enc.fit(X_cat)
X = enc.transform(X_cat)
test = enc.transform(test_cat)

In [9]:
#don't need stores with 0 sales
X = X[Y>0]
Y = Y[Y>0]
print np.shape(X)

(844338, 1178)


In [10]:
#log transform sales
print Y[:5]
Y = np.log(Y)
print Y[:5]

[  5263.   6064.   8314.  13995.   4822.]
[ 8.56845649  8.71012493  9.02569612  9.5464554   8.48094406]


In [11]:
X

<844338x1178 sparse matrix of type '<type 'numpy.float64'>'
	with 7599042 stored elements in Compressed Sparse Row format>

In [13]:
#Do some cross val testing
kf = KFold(np.shape(X)[0], n_folds=2)
i=0
rmspe=[]
t1=time.time()
for train_index, test_index in kf:
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    #Scale Data
    #Scale X
    scaler = preprocessing.StandardScaler(with_mean=False)
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    #train model
    #mod = RandomForestRegressor(n_estimators=50,n_jobs=-1) #very slow, especially on all data
    #mod = GradientBoostingRegressor(n_estimators=50)
    #mod = linear_model.SGDRegressor()#~500 error requires scaling
    #mod = naive_bayes.GaussianNB() #doesn't work with sparse
    mod.fit(X_train_scaled,Y_train)
    #make predictions
    preds = np.exp(mod.predict(X_test_scaled))
    Y_test = np.exp(Y_test)
    #score
    rmspe.append((np.mean(((preds-Y_test)/(Y_test+1))**2))**0.5)
    print i+1,rmspe[i]
    i=i+1
print 'Time =',int(time.time()-t1),'s'
print 'RMSPE avg =',np.mean(rmspe)

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

In [13]:
def xgboost(train_data, train_labels, test_data, test_labels):
    dtrain = xgb.DMatrix(train_data, label=train_labels)
    dtest = xgb.DMatrix(test_data, label=test_labels)
    param = {'objective':'reg:linear'}
    param['eval_metric']='auc'
    evallist = [(dtest,'eval'), (dtrain,'train')]

    num_round=10
    return xgb.train(param, dtrain, num_round, evallist)

In [12]:
#Hold back some test data from the train data
X[0]

<1x1178 sparse matrix of type '<type 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [ ]:

X_train, X_test = X[train_index], X[test_index]
Y_train, Y_test = Y[train_index], Y[test_index]
#Scale Data
#Scale X
scaler = preprocessing.StandardScaler(with_mean=False)
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
#train model
#mod = RandomForestRegressor(n_estimators=50,n_jobs=-1) #very slow, especially on all data
#mod = GradientBoostingRegressor(n_estimators=50)
#mod = linear_model.SGDRegressor()#~500 error requires scaling
#mod = naive_bayes.GaussianNB() #doesn't work with sparse
mod = xgboost(X_train, Y_train, X_test, Y_test)
mod.fit(X_train_scaled,Y_train)
#make predictions
preds = np.exp(mod.predict(X_test_scaled))
Y_test = np.exp(Y_test)
#score
rmspe.append((np.mean(((preds-Y_test)/(Y_test+1))**2))**0.5)
rmspe[i]

print 'Time =',int(time.time()-t1),'s'
print 'RMSPE avg =',np.mean(rmspe)

In [14]:
test_scaled = scaler.transform(test)
test_preds = np.exp( mod.predict(test_scaled) )

outF = open('sub3.csv','wb')
fwriter = csv.writer(outF,delimiter=',')
fwriter.writerow(['Id','Sales'])
for i in range(len(test_preds)):
    fwriter.writerow([i+1,int(test_preds[i])])
outF.close()